In [1]:
"""
用于裁剪、标注郴州医生图像数据
输出：原图、标注图、MASK图、ROI图像
"""

import cv2
import os
import io
import json
import numpy
import imageio
import mmcv

In [2]:
"""
检查文件，参数（目录，后缀）
"""

def find_file(directory, extension):
    print("[INFO] directory: " + directory)
    target_files = []
    for target_file in os.listdir(directory):
        if target_file.endswith(extension):
            print("[INFO] " + extension + "_file: " + directory + "/" + target_file)
            target_files += [target_file]
    target_files.sort()
    return target_files

In [3]:
def read_json_file(directory, json_folder_name, filename, output_directory):
    print("[INFO] read_json_file: " + directory + "/" + filename)
    prefix_filename = filename[:-5]
    print(prefix_filename)
    jpg_name = prefix_filename + ".jpg"
    output_jpg_name = jpg_name.replace('-','').replace('(','').replace(')','').replace('_','').replace(' ','')
    jpg_file = directory + "/" + jpg_name
    if os.path.exists(jpg_file):
        image = imageio.imread(jpg_file)
        with io.open(json_folder_name + "/" + filename,'r',encoding='gbk') as load_f:
            load_dict = json.load(load_f)
            annotation_image = create_image(jpg_file)
            box_image = create_image(jpg_file)
            height = image.shape[0]
            width  = image.shape[1]
            mask_image = numpy.zeros((height,width,3)).astype('uint8')
            mask_box_image = numpy.zeros((height,width,3)).astype('uint8')
            mask_circular_image = numpy.zeros((height,width,3)).astype('uint8')

            for sub_id, mark in enumerate(load_dict["shapes"]):
                a_image = create_image(jpg_file)
                b_image = create_image(jpg_file)
                label_name = mark["label"].replace(',','')
                # print(mark["points"])
                a = numpy.array(mark["points"], numpy.int32)
                cv2.fillConvexPoly(mask_image, a, (230,230,230))
                cv2.fillConvexPoly(b_image, a, (0, 0, 255))
                # cv2.polylines(annotation_image, a, True, color = (0, 0, 255), thickness = 1) # 图像，点集，是否闭合，颜色，线条粗细
                point_size = len(mark["points"])
                pre_point = mark["points"][point_size - 1]
                for points in mark["points"]:
                    cv2.line(annotation_image, (points[0], points[1]), (pre_point[0], pre_point[1]), color = (0, 0, 255), thickness = 5)
                    pre_point = points
                lmax = numpy.max(a[0:len(a)][:, 0])
                lmin = numpy.min(a[0:len(a)][:, 0])
                hmax = numpy.max(a[0:len(a)][:, 1])
                hmin = numpy.min(a[0:len(a)][:, 1])
#                 roi_image = image[hmin: hmax, lmin: lmax]
                # annotation_image = cv2.rectangle(annotation_image, (lmin, hmin), (lmax, hmax), color = (0, 0, 255), thickness = 1) # 图像，点集，是否闭合，颜色，线条粗细
#                 if label_name.startswith('A1'):
#                     box_image = cv2.putText(box_image, 'Cystic Nodule', (lmin - 10, hmin - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (127, 255, 0), 2)
#                     box_image = cv2.rectangle(box_image, (lmin - 10, hmin - 10), (lmax + 10, hmax + 10), color = (127, 255, 0), thickness = 3, lineType=cv2.LINE_AA) # 图像，点集，是否闭合，颜色，线条粗细0 255 127
#                 if label_name.startswith('A4'):
#                     box_image = cv2.putText(box_image, 'Solid Nodule', (lmin - 10, hmin - 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 245), 2)
#                     box_image = cv2.rectangle(box_image, (lmin - 10, hmin - 10), (lmax + 10, hmax + 10), color = (0, 0, 245), thickness = 3, lineType=cv2.LINE_AA) # 图像，点集，是否闭合，颜色，线条粗细
                if sub_id == 0:
                    box_image = cv2.rectangle(box_image, (lmin, hmin), (lmax, hmax), color = (0, 255, 0), thickness = 2, lineType=cv2.LINE_AA) # 图像，点集，是否闭合，颜色，线条粗细

                mask_box_image = mask_image.copy()
                mask_box_image = cv2.rectangle(mask_box_image, (lmin, hmin), (lmax, hmax), color = (0,255,0), thickness = 1, lineType=cv2.LINE_AA) # 图像，点集，是否闭合，颜色，线条粗细0 255 127

                hcenter = int((hmin + hmax) / 2)
                lcenter = int((lmin + lmax) / 2)
                roi_image = image[hcenter - 200: hcenter + 200, lcenter - 200: lcenter + 200].copy()
                mask_roi_image = mask_image[hcenter - 200: hcenter + 200, lcenter - 200: lcenter + 200].copy()
                test_image = cv2.addWeighted(image, 1, mask_image, 1, 0, dtype=cv2.CV_32F)
                test_image = cv2.rectangle(test_image, (lmin, hmin), (lmax, hmax), color = (255,0,0), thickness = 1, lineType=cv2.LINE_AA)
                mask_circular_image = numpy.zeros((height,width,3)).astype('uint8')
                a_image = cv2.ellipse(a_image, (lcenter, hcenter), (int((lmax - lmin)/2), int((hmax - hmin)/2)), 0, 0, 360, color=(100,149,237), thickness=-1, lineType=0) #画椭圆
#                 test_image = cv2.addWeighted(mask_image, 1, mask_circular_image, 0.9, 0, dtype=cv2.CV_32F)
#                 test_image = cv2.ellipse(test_image, (lcenter, hcenter), (int((lmax - lmin)/2), int((hmax - hmin)/2)), 0, 0, 360, color=(100,149,237), thickness=1, lineType=0) #画椭圆
#                 test_image = cv2.rectangle(mask_image, (lmin, hmin), (lmax, hmax), color = (0,230,0), thickness = 1, lineType=cv2.LINE_AA)
                test_roi_image = test_image[hcenter - 200: hcenter + 200, lcenter - 200: lcenter + 200].copy()
                
                save_jpg_file(output_directory + "/roi/" + label_name + "_" + str(sub_id) + "_" + output_jpg_name, mmcv.bgr2rgb(roi_image))
                save_jpg_file(output_directory + "/mask_roi/" + label_name + "_" + str(sub_id) + "_" + output_jpg_name, mmcv.bgr2rgb(mask_roi_image))
                save_jpg_file(output_directory + "/mix_roi/" + label_name + "_" + str(sub_id) + "_" + output_jpg_name, mmcv.bgr2rgb(test_roi_image))
                save_jpg_file(output_directory + "/mask_demo/" + label_name + "_" + str(sub_id) + "_" + output_jpg_name, mmcv.bgr2rgb(a_image))
                save_jpg_file(output_directory + "/mask_demo_gt/" + label_name + "_" + str(sub_id) + "_" + output_jpg_name, mmcv.bgr2rgb(b_image))

            save_jpg_file(output_directory + "/annotation/" + label_name + "_" + output_jpg_name, mmcv.bgr2rgb(annotation_image))
            save_jpg_file(output_directory + "/mask/" + label_name + "_" + output_jpg_name, mmcv.bgr2rgb( mask_image))
            save_jpg_file(output_directory + "/image/" + label_name + "_" + output_jpg_name, mmcv.bgr2rgb(image))
            save_jpg_file(output_directory + "/box/" + label_name + "_" + output_jpg_name, mmcv.bgr2rgb(box_image))
            save_jpg_file(output_directory + "/mask_box/" + label_name + "_" + output_jpg_name,  mmcv.bgr2rgb(mask_box_image))

In [4]:
def create_image(jpg_file):
    image = imageio.imread(jpg_file)
    height = image.shape[0]
    width = image.shape[1]
    new_image = cv2.resize(image, (width, height), interpolation=cv2.INTER_LINEAR)
    return new_image


In [5]:
def save_jpg_file(jpg_output_file, output_image):
    cv2.imwrite(jpg_output_file, output_image)

In [6]:
# 郴州数据目录
image_folder_name = '../../data/origin/chenzhou/image/'
json_folder_name = '../../data/origin/chenzhou/json/'
# 郴州数据输出目录
output_folder_name = '../../data/preprocess/demo/'
# 标注图像输出目录
if os.path.exists(output_folder_name + "/annotation/") == False:
    os.makedirs(output_folder_name + "/annotation/")
# mask图像输出目录
if os.path.exists(output_folder_name + "/mask/") == False:
    os.makedirs(output_folder_name + "/mask/")
# ROI图像输出目录
if os.path.exists(output_folder_name + "/mask_roi/") == False:
    os.makedirs(output_folder_name + "/mask_roi/")
    # ROI图像输出目录
if os.path.exists(output_folder_name + "/mask_box/") == False:
    os.makedirs(output_folder_name + "/mask_box/")
# ROI图像输出目录
if os.path.exists(output_folder_name + "/roi/") == False:
    os.makedirs(output_folder_name + "/roi/")
    # ROI图像输出目录
if os.path.exists(output_folder_name + "/mix_roi/") == False:
    os.makedirs(output_folder_name + "/mix_roi/")
# 原始图像输出目录
if os.path.exists(output_folder_name + "/box/") == False:
    os.makedirs(output_folder_name + "/box/")
# 原始图像输出目录
if os.path.exists(output_folder_name + "/image/") == False:
    os.makedirs(output_folder_name + "/image/")
# 原始图像输出目录
if os.path.exists(output_folder_name + "/mask_demo/") == False:
    os.makedirs(output_folder_name + "/mask_demo/")
    # 原始图像输出目录
if os.path.exists(output_folder_name + "/mask_demo_gt/") == False:
    os.makedirs(output_folder_name + "/mask_demo_gt/")
# json文件列表
# json_files = find_file(json_folder_name, '.json')
# json_files = ['13THY420180717154528804T.json', '39THY1420180720115201073T.json']
# json_files = ['98THY4320180820151729074.json', '107THY4620180823092511176T.json']
# json_files = ['55THY1920180723153752131T.json']

json_files = ['13THY420180717154528804T.json', 
              '182-12221420180907_THY131_20180907142255572T.json', 
              '98THY4320180820151729074.json', 
              '55THY1920180723153752131T.json', 
              '149-39310820180803_THY111_20180803083209135T.json', 
              '39THY1420180720115201073T.json', 
              '107THY4620180823092511176T.json', 
              '197-15021020181010_THY143_20181010100227611T.json',
              '138-04180920180801_THY1005_20180801092118768V.json', 
              '103THY4420180822155400562T.json', 
              '106THY4620180823092452358T.json',
              '48THY1820180723105605185.json',
              '191-27090920180913_THY136_20180913091148742T.json',
             '69THY2520180725160205947T.json',
             '131THY5620180830155731310T.json',
             '164-35091520180815_THY120_20180815151120041T.json',
             '16THY520180717164323160V.json',
             '207-37380920181105_THY152_20181105093920028T.json',
             '153-03221020180803_THY113_20180803102243041T.json',
             '174-48191420180905_THY126_20180905142132295V.json',
             '121THY5220180830104301820T.json',
             '18THY620180718085559429V.json',
             '76THY31YAJIAYAN20180730095813902T.json',
             '193-05161420180913_THY138_20180913141631443V.json',
             '101THY4320180820151937778T.json']

for json_file in json_files:
    read_json_file(image_folder_name, json_folder_name, json_file, output_folder_name)

# binary_map = image * 255

[INFO] read_json_file: ../../data/origin/chenzhou/image//13THY420180717154528804T.json
13THY420180717154528804T
[INFO] read_json_file: ../../data/origin/chenzhou/image//182-12221420180907_THY131_20180907142255572T.json
182-12221420180907_THY131_20180907142255572T
[INFO] read_json_file: ../../data/origin/chenzhou/image//98THY4320180820151729074.json
98THY4320180820151729074
[INFO] read_json_file: ../../data/origin/chenzhou/image//55THY1920180723153752131T.json
55THY1920180723153752131T
[INFO] read_json_file: ../../data/origin/chenzhou/image//149-39310820180803_THY111_20180803083209135T.json
149-39310820180803_THY111_20180803083209135T
[INFO] read_json_file: ../../data/origin/chenzhou/image//39THY1420180720115201073T.json
39THY1420180720115201073T
[INFO] read_json_file: ../../data/origin/chenzhou/image//107THY4620180823092511176T.json
107THY4620180823092511176T
[INFO] read_json_file: ../../data/origin/chenzhou/image//197-15021020181010_THY143_20181010100227611T.json
197-15021020181010_TH